In [1]:
from datetime import datetime
import pytz

def fecha_hora_lima():
    lima_tz = pytz.timezone('America/Lima')
    lima_time = datetime.now(lima_tz)
    return lima_time.strftime('%Y%m%d-%H%M%S')

def fecha_hora_lima_db():
    lima_tz = pytz.timezone('America/Lima')
    lima_time = datetime.now(lima_tz)
    return lima_time.strftime('%Y-%m-%d %H:%M:%S')

fecha_hora_para_archivos = fecha_hora_lima()
fecha_hora_para_base_de_datos = fecha_hora_lima_db()

In [2]:
import pandas as pd
import requests
import json

# URL del JSON
url = 'https://www.plazavea.com.pe/api/catalog_system/pub/category/tree/3'

# Solicitar el contenido JSON desde la URL
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    data = response.json()  # Convertir la respuesta en un objeto JSON
    file_name = f'categories_{fecha_hora_para_archivos}.json'
    
    # Guardar el JSON en un archivo
    with open(file_name, 'w') as file:
        json.dump(data, file, indent=4)

    print('El archivo JSON se ha guardado exitosamente.')
    with open(file_name, 'r') as file:
        data = json.load(file)
    print(len(data)) 
else:
    print(f'Error al obtener el JSON. Código de estado: {response.status_code}')  

El archivo JSON se ha guardado exitosamente.
37


In [ ]:
# file_path = 'plazavea_categories.json'
# 
# with open(file_path, 'r') as file:
#     data = json.load(file)
#     print(json.dumps(data, indent=4))

In [3]:
columns = ['id', 'name', 'hasChildren', 'url', 'NumChildren', 'Title', 'MetaTagDescription', 'parent_id']
df = pd.DataFrame(columns=columns)
df

,id,name,hasChildren,url,NumChildren,Title,MetaTagDescription,parent_id


In [4]:
for item in data:
  if item['hasChildren']:
    item['parent_id'] = ''
    item['NumChildren'] = len(item['children'])

    df.loc[len(df)] = item

    parent_id = item['id']

    for subitem in item['children']:
      subitem['parent_id'] = parent_id
      sub_parent_id = subitem['id']
      subitem['NumChildren'] = len(subitem['children'])

      df.loc[len(df)] = subitem

      if len(subitem['children']) > 0:
        for subsubitem in subitem['children']:
          subsubitem['parent_id'] = sub_parent_id
          subsubitem['NumChildren'] = len(subsubitem['children'])
          df.loc[len(df)] = subsubitem

In [5]:
df.query("id == 952")

,id,name,hasChildren,url,NumChildren,Title,MetaTagDescription,parent_id
4,952,Agua Tónica,False,https://www.plazavea.com.pe/bebidas/aguas/agua...,0,Agua Tonica | plazaVea,Agua Tonica en oferta y al mejor precio de ven...,6


In [6]:
file_name_categories_csv = f'plazavea_categories_{fecha_hora_para_archivos}.csv'
df.to_csv(file_name_categories_csv, index=False)

In [7]:
cats = df.query("parent_id == ''")['id'].to_list()
print(cats)

[2, 297, 399, 620, 431, 679, 77, 681, 898, 210, 621, 1138, 493, 666, 814, 845, 922, 1098, 1105, 478, 1046, 1073, 1383, 1503, 678, 1409, 1765, 1767, 2331, 2941, 3013, 3072, 3086, 3174, 3201, 3253, 4180]


In [8]:
import time

products = []

for cat in cats:
    start = 0
    end = 49
    
    while True:
        time.sleep(1)
        url = f"https://www.plazavea.com.pe/api/catalog_system/pub/products/search?fq=C:{cat}&_from={start}&_to={end}&O=OrderByPriceDESC"
        
        response = requests.get(url)
        
        if response.status_code == 200 or response.status_code == 206:
            data = response.json()
            if not data:
                break
                 
            products.extend(data)
            start += 50
            end += 50
        else:
            break
    
    print(f"status_code = {response.status_code}")
    print(f"cat = {cat}, start = {start}, end = {end}")

 
file_name_productos_csv = f'productos_{fecha_hora_para_archivos}.json'

with open(file_name_productos_csv, 'w') as file:
    json.dump(products, file, indent=4)

status_code = 200
cat = 2, start = 650, end = 699
status_code = 400
cat = 297, start = 2550, end = 2599
status_code = 400
cat = 399, start = 2550, end = 2599
status_code = 400
cat = 620, start = 2550, end = 2599
status_code = 500
cat = 431, start = 850, end = 899
status_code = 500
cat = 679, start = 2100, end = 2149
status_code = 200
cat = 77, start = 650, end = 699
status_code = 500
cat = 681, start = 600, end = 649
status_code = 500
cat = 898, start = 1600, end = 1649
status_code = 200
cat = 210, start = 450, end = 499
status_code = 200
cat = 621, start = 800, end = 849
status_code = 500
cat = 1138, start = 150, end = 199
status_code = 500
cat = 493, start = 450, end = 499
status_code = 500
cat = 666, start = 1950, end = 1999
status_code = 200
cat = 814, start = 500, end = 549
status_code = 200
cat = 845, start = 600, end = 649
status_code = 400
cat = 922, start = 2550, end = 2599
status_code = 400
cat = 1098, start = 2550, end = 2599
status_code = 500
cat = 1105, start = 200, end = 

In [9]:
print(len(products))

41304


In [10]:
columns = ['productId', 'productName', 'brand', 'brandId', 'brandImageUrl','categoryId',
           'productTitle', 'metaTagDescription', 'categories', 'categoriesIds', 'link', 'Vendido por',
           'Descripción del producto', 'Presentación', 'Presentación/Empaque', 'Pack', 'Porción Sugerida',
           'Azúcares Por Porción', 'Carbohidratos Por Porción', 'Descripción Nutricional', 'Grasas Por Porción',
           'Advertencias de consumo', 'itemId', 'ean', 'imageUrl', 'Value', 'IsAvailable', 'AvailableQuantity',
           'Price', 'ListPrice', 'PriceWithoutDiscount', 'PriceValidUntil', 'Created']

df = pd.DataFrame(columns=columns)
df

,productId,productName,brand,brandId,brandImageUrl,categoryId,productTitle,metaTagDescription,categories,categoriesIds,...,ean,imageUrl,Value,IsAvailable,AvailableQuantity,Price,ListPrice,PriceWithoutDiscount,PriceValidUntil,Created


In [11]:
for product in products:
  print(product['items'][0]['sellers'][0]['commertialOffer']['Installments'][0]['Value'])
  print(product['items'][0]['sellers'][0]['commertialOffer']['PriceValidUntil'])
  print(product['items'][0]['sellers'][0]['commertialOffer']['ListPrice'])
  print(product['items'][0]['sellers'][0]['commertialOffer']['PriceWithoutDiscount'])
  break

57.9
3000-01-02T00:00:00Z
57.9
57.9


In [12]:
print(json.dumps(products[3099], indent=4))

{
    "productId": "100859353",
    "productName": "Crema Corporal Numae 110g + 3 Crema depiladora Lampi\u00f1a 120g",
    "brand": "LAMPI\u00d1A",
    "brandId": 40230196,
    "brandImageUrl": null,
    "linkText": "crema-corporal-numae-110g-3-crema-depiladora-lampina-120g-100859353",
    "productReference": "-",
    "productReferenceCode": "-",
    "categoryId": "2852",
    "productTitle": "Crema Corporal Numae 110g + 3 Crema depiladora Lampi\u00f1a 120g | plazaVea",
    "metaTagDescription": "\u00a1Compra Online Crema Corporal Numae 110g + 3 Crema depiladora Lampi\u00f1a 120g desde donde est\u00e9s en plazaVea.com.pe!",
    "releaseDate": "2023-10-21T00:00:00Z",
    "clusterHighlights": {},
    "productClusters": {},
    "searchableClusters": {},
    "categories": [
        "/Cuidado Personal y Salud/Depilaci\u00f3n/Cremas para Depilar/",
        "/Cuidado Personal y Salud/Depilaci\u00f3n/",
        "/Cuidado Personal y Salud/"
    ],
    "categoriesIds": [
        "/297/2849/2852/"

In [ ]:
# print(json.dumps(products, indent=4))

In [14]:
def print_value(value, key):
    if value:
        print(f"{key} = {value}")
    else:
        print(f"{key} = None")

In [15]:
from bs4 import BeautifulSoup

def clean_html(html_code):
    if html_code is None:
        return ''
    if isinstance(html_code, list):
        html_code = ' '.join(html_code)
    soup = BeautifulSoup(html_code, 'html.parser')
    return soup.get_text()

In [16]:
def get_product_data(product, fecha_db):
    product['Created'] = fecha_db
    new_product = {}
    new_product['productId'] = product.get('productId')
    new_product['productName'] = product.get('productName')
    new_product['brand'] = product.get('brand')
    new_product['brandId'] = product.get('brandId')
    new_product['brandImageUrl'] = product.get('brandImageUrl')
    new_product['categoryId'] = product.get('categoryId')
    new_product['productTitle'] = product.get('productTitle')
    new_product['metaTagDescription'] = product.get('metaTagDescription')
    new_product['categories'] = product.get('categories')
    new_product['categoriesIds'] = product.get('categoriesIds')
    new_product['link'] = product.get('link')
    new_product['Vendido por'] = product.get('Vendido por')
    new_product['Descripción del producto'] = clean_html(product.get('Descripción del producto'))
    new_product['Presentación'] = product.get('Presentación')
    new_product['Presentación/Empaque'] = product.get('Presentación/Empaque')
    new_product['Pack'] = product.get('Pack')
    new_product['Porción Sugerida'] = product.get('Porción Sugerida')
    new_product['Azúcares Por Porción'] = product.get('Azúcares Por Porción')
    new_product['Carbohidratos Por Porción'] = product.get('Carbohidratos Por Porción')
    new_product['Descripción Nutricional'] = product.get('Descripción Nutricional')
    new_product['Grasas Por Porción'] = product.get('Grasas Por Porción')
    new_product['Advertencias de consumo'] = product.get('Advertencias de consumo')
    
    if product['items']:
        new_product['itemId'] = product['items'][0].get('itemId', None)
        new_product['ean'] = product['items'][0].get('ean', None)
        if product['items'][0]['images']:
            new_product['imageUrl'] = product['items'][0]['images'][0].get('imageUrl', None)
            if product['items'][0]['sellers']:
                commertial_offer = product['items'][0]['sellers'][0].get('commertialOffer', {})
                if commertial_offer.get('Installments'):
                    new_product['Value'] = commertial_offer['Installments'][0].get('Value', None)
                new_product['IsAvailable'] = commertial_offer.get('IsAvailable', None)
                new_product['AvailableQuantity'] = commertial_offer.get('AvailableQuantity', None)
                new_product['Price'] = commertial_offer.get('Price', None)
                new_product['ListPrice'] = commertial_offer.get('ListPrice', None)
                new_product['PriceWithoutDiscount'] = commertial_offer.get('PriceWithoutDiscount', None)
                new_product['PriceValidUntil'] = commertial_offer.get('PriceValidUntil', None)

    new_product['Created'] = product['Created']
    
    return new_product

In [17]:
# Para validar que los datos se están extrayendo correctamente

i = 0
for product in products:
    i += 1
    product['Created'] = fecha_hora_lima_db()
    print_value(product.get('productId'), 'productId')
    print_value(product.get('productName'), 'productName')
    print_value(product.get('brand'), 'brand')
    print_value(product.get('brandId'), 'brandId')
    print_value(product.get('brandImageUrl'), 'brandImageUrl')
    print_value(product.get('categoryId'), 'categoryId')
    print_value(product.get('productTitle'), 'productTitle')
    print_value(product.get('metaTagDescription'), 'metaTagDescription')
    print_value(product.get('categories'), 'categories')
    print_value(product.get('categoriesIds'), 'categoriesIds')
    print_value(product.get('link'), 'link')
    print_value(product.get('Vendido por'), 'Vendido por')
    print_value(clean_html(product.get('Descripción del producto')), 'Descripción del producto')
    print_value(product.get('Presentación'), 'Presentación')
    print_value(product.get('Presentación/Empaque'), 'Presentación/Empaque')
    print_value(product.get('Pack'), 'Pack')
    print_value(product.get('Porción Sugerida'), 'Porción Sugerida')
    print_value(product.get('Azúcares Por Porción'), 'Azúcares Por Porción')
    print_value(product.get('Carbohidratos Por Porción'), 'Carbohidratos Por Porción')
    print_value(product.get('Descripción Nutricional'), 'Descripción Nutricional')
    print_value(product.get('Grasas Por Porción'), 'Grasas Por Porción')
    print_value(product.get('Advertencias de consumo'), 'Advertencias de consumo')
    print_value(product['items'][0]['itemId'], 'itemId')
    print_value(product['items'][0]['ean'], 'ean')
    print_value(product['items'][0]['images'][0]['imageUrl'], 'imageUrl')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['Installments'][0]['Value'], 'Value')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['IsAvailable'], 'IsAvailable')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['AvailableQuantity'], 'AvailableQuantity')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['Price'], 'Price')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['ListPrice'], 'ListPrice')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['PriceWithoutDiscount'], 'PriceWithoutDiscount')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['PriceValidUntil'], 'PriceValidUntil')
    print_value(product['Created'], 'Created')
    
    if i == 3:
        break

productId = 24015
productName = Agua Tónica BRITVIC Paquete 12un Lata 150ml
brand = BRITVIC
brandId = 66825
brandImageUrl = None
categoryId = 952
productTitle = Agua Tónica BRITVIC Paquete 12un Lata 150ml | plazaVea
metaTagDescription = ¡Compra Online Agua Tónica BRITVIC Paquete 12un Lata 150ml desde donde estés en plazaVea.com.pe!
categories = ['/Bebidas/Aguas/Agua Tónica/', '/Bebidas/Aguas/', '/Bebidas/']
categoriesIds = ['/2/6/952/', '/2/6/', '/2/']
link = https://www.plazavea.com.pe/agua-tonica-britvic-paquete-12un-x-lata-150ml/p
Vendido por = ['plazaVea']
Descripción del producto = 



Importante destacarAgua tónica Marca: BRITVIC  Contenido: 12un de 150ml  




Descripción del productoAGUAS
plazaVea presenta una gran variedad de Bebidas para que elijas tu favorita de acuerdo a diferentes situaciones u ocasiones especiales. Hidratarse es muy importante para el correcto cuidado de nuestro organismo, además que nos ayuda a lucir mejor, porque una piel hidratada se refleja siempre en

In [18]:
for product in products:
  new_product = get_product_data(product, fecha_hora_para_base_de_datos)
  df.loc[len(df)] = new_product

In [19]:
df.sample(n=5)

,productId,productName,brand,brandId,brandImageUrl,categoryId,productTitle,metaTagDescription,categories,categoriesIds,...,ean,imageUrl,Value,IsAvailable,AvailableQuantity,Price,ListPrice,PriceWithoutDiscount,PriceValidUntil,Created
28578,101025107,Cuna Corral Mecedora Ebaby Sally EB716 Azul,EBABY,73923,None,657,Cuna Corral Mecedora Ebaby Sally EB716 Azul | ...,¡Compra online Cuna Corral Mecedora Ebaby Sall...,[/Bebé e Infantil/Dormitorio Bebé e Infantil/C...,"[/1503/1435/657/, /1503/1435/, /1503/]",...,,https://plazavea.vteximg.com.br/arquivos/ids/2...,379.0,True,10,379.0,499.0,379.0,2030-01-31T00:00:00Z,2024-10-08 15:28:01
38416,100827504,Autoradio Pioneer DEH S4250BT,PIONEER,60617,None,3261,Autoradio Pioneer DEH S4250BT | plazaVea,¡Compra online Autoradio Pioneer DEH S4250BTde...,"[/Automotriz/Accesorios para Auto/Autoradios/,...","[/3253/3254/3261/, /3253/3254/, /3253/]",...,,https://plazavea.vteximg.com.br/arquivos/ids/2...,620.0,True,10,620.0,620.0,620.0,None,2024-10-08 15:28:01
3939,101132849,Cepillo Manual de Pelusas Amarillo Y+Post-it a...,RYBIU IMPORT,40228561,None,418,None,Si quieres mantener tus prendas libre de pelos...,[/Limpieza/Cuidado de la Ropa/Accesorios para ...,"[/399/400/418/, /399/400/, /399/]",...,,https://plazavea.vteximg.com.br/arquivos/ids/2...,39.9,True,50,39.9,57.0,39.9,2029-08-22T00:00:00Z,2024-10-08 15:28:01
18861,101139549,Carro a Batería Chevrolet Daewon con Pantalla ...,IMPORTADO,40230546,None,1584,None,¡Transforma cada paseo en una emocionante aven...,[/Juguetes y Juegos/Juguetería/Juguetes a Bate...,"[/922/923/1584/, /922/923/, /922/]",...,,https://plazavea.vteximg.com.br/arquivos/ids/2...,2839.0,True,10,2839.0,5599.0,2839.0,2029-12-31T00:00:00Z,2024-10-08 15:28:01
22375,100010568,Juego de Sala 3 2 Denieke Beige,ALTO HOGAR,40223493,None,1120,Juego de Sala 3 2 Denieke Beige | plazaVea,¡Compra online Juego de Sala 3 2 Denieke Azul ...,"[/Muebles/Muebles de Sala/Juegos de sala/, /Mu...","[/1098/1030/1120/, /1098/1030/, /1098/, /1765/...",...,30012124,https://plazavea.vteximg.com.br/arquivos/ids/1...,3499.0,True,13,3499.0,3800.0,3499.0,2028-01-01T00:00:00,2024-10-08 15:28:01


In [27]:
df[['productId', 'productName', 'brand', 'categoryId', 'link', 'Price', 'AvailableQuantity']].sample(n=8)

,productId,productName,brand,categoryId,link,Price,AvailableQuantity
2295,100442979,Aminoácidos Level Pro Bcaa Level 8000 1000gr ...,LEVEL PRO,1246,https://www.plazavea.com.pe/aminoacidos-level-...,189.90,999
25562,100796574,Crema de Ajo FERYMAR Envase 250g,FERYMAR,1681,https://www.plazavea.com.pe/crema-de-ajo-ferym...,12.90,134
5478,100099486,Limpiador POETT Bebé Botella 1.8L,POETT,424,https://www.plazavea.com.pe/limpiador-poett-be...,7.50,81
9506,100982722,Aire Acondicionado Split Khöne Inverter KMI-18...,KHÖNE,745,https://www.plazavea.com.pe/aire-acondicionado...,2899.00,29
32491,100280567,Dormitorio 3 Cajones PARAISO Royal Elizabeth C...,PARAÍSO,3550,https://www.plazavea.com.pe/dormitorio-3-cajon...,1599.00,1000000
17555,100629088,Libreta Moustachine Classic Leather Look Negro...,MOUSTACHINE,904,https://www.plazavea.com.pe/libreta-moustachin...,49.90,20
36081,101017636,"Reloj Invicta 43929 Blanco, Titanio Hombres",INVICTA,3049,https://www.plazavea.com.pe/reloj-invicta-4392...,2699.00,3
11690,62885,Jugo Natural ECOFRESH Naranja Botella 1.8L,ECOFRESH,1764,https://www.plazavea.com.pe/jugo-natural-ecofr...,24.99,0


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41304 entries, 0 to 41303
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   productId                  41304 non-null  object 
 1   productName                41304 non-null  object 
 2   brand                      41304 non-null  object 
 3   brandId                    41304 non-null  int64  
 4   brandImageUrl              26 non-null     object 
 5   categoryId                 41304 non-null  object 
 6   productTitle               40729 non-null  object 
 7   metaTagDescription         41304 non-null  object 
 8   categories                 41304 non-null  object 
 9   categoriesIds              41304 non-null  object 
 10  link                       41304 non-null  object 
 11  Vendido por                41300 non-null  object 
 12  Descripción del producto   41304 non-null  object 
 13  Presentación               2722 non-null   object 


In [28]:
df['product_uri_json'] = df['productId'].apply(lambda x: f'https://www.plazavea.com.pe/api/catalog_system/pub/products/search?fq=productId:{x}')

In [29]:
df['product_uri_json'].sample(n=5)

26955    https://www.plazavea.com.pe/api/catalog_system...
14844    https://www.plazavea.com.pe/api/catalog_system...
34448    https://www.plazavea.com.pe/api/catalog_system...
18717    https://www.plazavea.com.pe/api/catalog_system...
25050    https://www.plazavea.com.pe/api/catalog_system...
Name: product_uri_json, dtype: object

In [30]:
file_name_productos_csv = f'plazavea-productos_{fecha_hora_para_archivos}.csv'
df.to_csv(file_name_productos_csv, index=False)